# Time Frequency Analysis of Percept recordings

## 0. Loading packages and functions, defining paths

In [1]:
# Importing Python and external packages
import os
import sys
import importlib
import json
from dataclasses import dataclass, field, fields
from itertools import compress
import csv
import pandas as pd
import numpy as np

import scipy
import matplotlib.pyplot as plt
from scipy import signal

import openpyxl
from openpyxl import Workbook, load_workbook
import xlrd

#mne
import mne_bids
import mne


# additionally imported from Varvara
# from importlib import reload          # is this necessary?

# from scipy.signal import spectrogram, hanning     # is this necessary?

# from mne.time_frequency import tfr_morlet         # is this necessary?

In [2]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('mne_bids', mne_bids.__version__)
print('mne', mne.__version__)
print('sci-py', scipy.__version__)

Python sys 3.10.6 | packaged by conda-forge | (main, Oct 24 2022, 16:02:16) [MSC v.1916 64 bit (AMD64)]
pandas 1.4.4
numpy 1.23.3
mne_bids 0.11.1
mne 1.2.1
sci-py 1.9.3


In [3]:
# create a path to the PyPerceive_Project folder -> project_path
# and a path to the "Data" folder and "Code" folder within the project
project_path = os.getcwd()
while project_path[-18:] != 'PyPerceive_Project':
    project_path = os.path.dirname(project_path)
    
data_path = os.path.join(project_path, 'Data')
code_path = os.path.join(project_path, 'Code', 'PyPerceive', 'code')

project_path, data_path, code_path
# change directory to code path???
os.chdir(code_path)
os.getcwd()

'c:\\Users\\jebe12\\Research\\PyPerceive_Project\\Code\\PyPerceive\\code'

## 1. Load Data using MainClass2

In [4]:
import PerceiveImport.classes.MainClass2 as mainclass
importlib.reload(mainclass)

<module 'PerceiveImport.classes.MainClass2' from 'c:\\Users\\jebe12\\Research\\PyPerceive_Project\\Code\\PyPerceive\\code\\PerceiveImport\\classes\\MainClass2.py'>

In [8]:
# define an example instance and fill in the values of the dataclass PerceiveData 
# choose the values you are interested in analyzing further

sub017 = mainclass.PerceiveData(
    sub = "017", 
    incl_modalities=["Survey"],
    incl_session = ["FU12M"],
    incl_condition =["M0S0"],
    incl_task = ["RestBSSuRingR"]
    )

# Funktion:
# Eingabe nur einzelner Strings, nicht mehrfacher Str in Liste
# bei keiner Eingabe der letzten attributes, wird default aus allen Möglichkeiten herausgegeben
# funktioniert nur bei vollständiger metadata Tabelle und nur, wenn filenames in perceiveFilenames == filenames in directory (nicht immer der Fall, vor allem der Anfang sub-017. sub017)
# d.h. metadata vorher überprüfen: neue xlsx mit matfiles in directory + paths (paths enthalten Info zB 3MFU) -> händisch copy paste in metadata Tabelle und vervollständigen
# gezielte Definition bereits in PerceveData Class -> 

# Pitfalls:
# bei mehreren Eingaben, funktioniert immer nur der 1.Input -> zB incl_session = ["FU3M", "PostOp", "FU12M"] -> nur FU3M funktioniert, Rest ist leer
# wenn nichts eingegeben wurde, funktioniert immer nur der 1.Input der Default Liste! -> zB default Liste ["PostOp", "FU3M", ] -> nur PostOp funktioniert, Rest ist leer
# missing function: if one attribute is left out, the connection to the next Class is missing ...  
# missing function: can´t select multiple inputs simultaneously, optimal would be: input List -> outcome: concatenated dataframes of multiple DF
# load mne raw: wenn man eine Liste mit paths lädt, kann man einen einzelnen path nicht einer metadata Tabelle zuordnen, weil mit os.walk wird eine andere Reihenfolge gelistet als in der metadata Tabelle

# neue Ziele: 
# matpath_list bestenfalls in zugehöriger Reihe in DataFrame als neues column hinzufügen -> sodass nur 1 DF weitergegeben wird
# self.data ist nur von taskClass abrufbar, optimal wäre es auch von Main, modality, session und condition class self.data abrufen zu können, und raw

c:\Users\jebe12\anaconda3\envs\pyPerceive_dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


MetadataClass has been loaded
Creating RawArray with float64 data, n_channels=6, n_times=5288
    Range : 0 ... 5287 =      0.000 ...    21.148 secs
Ready.


c:\Users\jebe12\anaconda3\envs\pyPerceive_dev\lib\site-packages\pymatreader\utils.py:230: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
c:\Users\jebe12\Research\PyPerceive_Project\Code\PyPerceive\code\PerceiveImport\methods\metadata_methods.py:154: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  data["raw_{0}".format(count)] = mne.io.read_raw_fieldtrip(
c:\Users\jebe12\Research\PyPerceive_Project\Code\PyPerceive\code\PerceiveImport\methods\metadata_methods.py:154: RuntimeWarning: Cannot guess the correct type of channel LFP_R_1A1B_STN. Making it a MISC channel.
  data["raw_{0}".format(count)] = mne.io.read_raw_fieldtrip(
c:\Users\jebe12\Research\PyPerceive_Project\Code\PyPerceive\code\PerceiveImport\methods\metadata_methods.

In [9]:
# Explore MainClass2:
rawsub017 = sub017.Survey.FU12M.metadata_selection
rawsub017


,report,perceiveFilename,task,session,condition,stimState,run,badChannels,maxAmp,stimPeriodRestZeroPre,...,stimPeriodRestTwo,stimPeriodRestThree,stimPeriodRestMax,stimPeriodRestZeroPost,stimPeriodMovZero,stimPeriodMovOne,stimPeriodMovTwo,stimPeriodMovThree,stimPeriodMovMax,path_to_perceive
0,Report_Json_Session_Report_20220328T144952.json,sub-20210330PSTN_ses-1951031311000087_run-LMTD...,RestBSSuRingL,FU12M,M1S0,Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
1,Report_Json_Session_Report_20220328T144952.json,sub-20210330PSTN_ses-1951031311000087_run-LMTD...,RestBSSuSegmIntraL,FU12M,M1S0,Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
2,Report_Json_Session_Report_20220328T144952.json,sub-20210330PSTN_ses-1951031311000087_run-LMTD...,RestBSSuSegmInterL,FU12M,M1S0,Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
3,Report_Json_Session_Report_20220328T144952.json,sub-20210330PSTN_ses-1951031311000087_run-LMTD...,RestBSSuRingR,FU12M,M1S0,Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
4,Report_Json_Session_Report_20220328T144952.json,sub-20210330PSTN_ses-1951031311000087_run-LMTD...,RestBSSuSegmIntraR,FU12M,M1S0,Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
5,Report_Json_Session_Report_20220328T144952.json,sub-20210330PSTN_ses-1951031311000087_run-LMTD...,RestBSSuSegmInterR,FU12M,M1S0,Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
6,Report_Json_Session_Report_20220318T124233.json,sub-20210330PSTN_ses-2022031311000088_run-LMTD...,RestBSSuRingL,FU12M,M0S0,Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
7,Report_Json_Session_Report_20220318T124233.json,sub-20210330PSTN_ses-2022031311000088_run-LMTD...,RestBSSuSegmIntraL,FU12M,M0S0,Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
8,Report_Json_Session_Report_20220318T124233.json,sub-20210330PSTN_ses-2022031311000088_run-LMTD...,RestBSSuSegmInterL,FU12M,M0S0,Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
9,Report_Json_Session_Report_20220318T124233.json,sub-20210330PSTN_ses-2022031311000088_run-LMTD...,RestBSSuRingR,FU12M,M0S0,Off,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...


In [11]:
rawsub017.info.keys() #info["ch_names"]

AttributeError: 'dict' object has no attribute 'info'

In [14]:
# Beispiel: ich möchte von sub 017, alle Survey files vom M0S0 für jeden Timepoint (session) 
# erstelle je 1 DF für jede session

sub017_PostOp = mainclass.PerceiveData(
    sub = "017", 
    incl_modalities=["Survey"],
    incl_session = ["PostOp"],
    incl_condition =["M0S0"],
    #incl_task = ["RestBSSuRingR", "RestBSSuRingL"]
    )

sub017_FU3M = mainclass.PerceiveData(
    sub = "017", 
    incl_modalities=["Survey"],
    incl_session = ["FU3M"],
    incl_condition =["M0S0"],
    #incl_task = ["RestBSSuRingR", "RestBSSuRingL"]
    )

sub017_FU12M = mainclass.PerceiveData(
    sub = "017", 
    incl_modalities=["Survey"],
    incl_session = ["FU12M"],
    incl_condition =["M0S0"],
    #incl_task = ["RestBSSuRingR", "RestBSSuRingL"]
    )    

sub017_DF = {
    "017PostOp": sub017_PostOp.Survey.PostOp.M0S0.metadata_selection, 
    "017FU3M": sub017_FU3M.Survey.FU3M.M0S0.metadata_selection, 
    "017FU12M": sub017_FU12M.Survey.FU12M.M0S0.metadata_selection, 
}

sub017_paths = {
    "017PostOp":sub017_PostOp.Survey.PostOp.M0S0.matpath_list,
    "017FU3M": sub017_FU3M.Survey.FU3M.M0S0.matpath_list,
    "017FU12M": sub017_FU12M.Survey.FU12M.M0S0.matpath_list
}


sub017_DF["017FU3M"]

c:\Users\jebe12\anaconda3\envs\pyPerceive_dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


MetadataClass has been loaded


c:\Users\jebe12\anaconda3\envs\pyPerceive_dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jebe12\anaconda3\envs\pyPerceive_dev\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


MetadataClass has been loaded
MetadataClass has been loaded


,report,perceiveFilename,task,session,condition,stimState,run,badChannels,maxAmp,stimPeriodRestZeroPre,...,stimPeriodRestTwo,stimPeriodRestThree,stimPeriodRestMax,stimPeriodRestZeroPost,stimPeriodMovZero,stimPeriodMovOne,stimPeriodMovTwo,stimPeriodMovThree,stimPeriodMovMax,path_to_perceive
0,sub-017_Report_Json_Session_Report_20210701T15...,sub017_ses-1951031311000097_run-LMTD2021070107...,RestBSSuRingL,FU3M,M0S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
1,sub-017_Report_Json_Session_Report_20210701T15...,sub017_ses-1951031311000097_run-LMTD2021070107...,RestBSSuSegmIntraL,FU3M,M0S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
2,sub-017_Report_Json_Session_Report_20210701T15...,sub017_ses-1951031311000097_run-LMTD2021070107...,RestBSSuSegmInterL,FU3M,M0S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
3,sub-017_Report_Json_Session_Report_20210701T15...,sub017_ses-1951031311000097_run-LMTD2021070107...,RestBSSuRingR,FU3M,M0S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
4,sub-017_Report_Json_Session_Report_20210701T15...,sub017_ses-1951031311000097_run-LMTD2021070107...,RestBSSuSegmIntraR,FU3M,M0S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...
5,sub-017_Report_Json_Session_Report_20210701T15...,sub017_ses-1951031311000097_run-LMTD2021070107...,RestBSSuSegmInterR,FU3M,M0S0,Off,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c:\Users\jebe12\Research\Longterm_beta_project...


In [45]:
# load the chosen perceive .mat files with MNE
# e.g.: from subject 017 all Survey files from FU3M 
# store each loaded raw file in a dictionary with keys named raw_idx -> idx unfortunately doesn´t match with DF because paths are in different order (initially looped through folders...)

raw_017FU3M = {}
count = 0

for file in sub017_paths["017FU3M"]:  # matselection stores a list of the selected .mat paths 
    
    count +=1

    raw_017FU3M["raw_{0}".format(count)] = mne.io.read_raw_fieldtrip(
        file,
        info={},
        data_name='data'
        )

Creating RawArray with float64 data, n_channels=6, n_times=5288
    Range : 0 ... 5287 =      0.000 ...    21.148 secs
Ready.
Creating RawArray with float64 data, n_channels=6, n_times=5288
    Range : 0 ... 5287 =      0.000 ...    21.148 secs
Ready.
Creating RawArray with float64 data, n_channels=3, n_times=5250
    Range : 0 ... 5249 =      0.000 ...    20.996 secs
Ready.


c:\Users\jebe12\anaconda3\envs\pyPerceive_dev\lib\site-packages\pymatreader\utils.py:230: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
C:\Users\jebe12\AppData\Local\Temp\ipykernel_20540\96999579.py:12: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  raw_017FU3M["raw_{0}".format(count)] = mne.io.read_raw_fieldtrip(
C:\Users\jebe12\AppData\Local\Temp\ipykernel_20540\96999579.py:12: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_03. Making it a MISC channel.
  raw_017FU3M["raw_{0}".format(count)] = mne.io.read_raw_fieldtrip(
C:\Users\jebe12\AppData\Local\Temp\ipykernel_20540\96999579.py:12: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_13. Making it a MISC channel.
  raw_017FU3M["raw_{0

Creating RawArray with float64 data, n_channels=6, n_times=5288
    Range : 0 ... 5287 =      0.000 ...    21.148 secs
Ready.
Creating RawArray with float64 data, n_channels=6, n_times=5288
    Range : 0 ... 5287 =      0.000 ...    21.148 secs
Ready.
Creating RawArray with float64 data, n_channels=3, n_times=5312
    Range : 0 ... 5311 =      0.000 ...    21.244 secs
Ready.


C:\Users\jebe12\AppData\Local\Temp\ipykernel_20540\96999579.py:12: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  raw_017FU3M["raw_{0}".format(count)] = mne.io.read_raw_fieldtrip(
C:\Users\jebe12\AppData\Local\Temp\ipykernel_20540\96999579.py:12: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_R_03. Making it a MISC channel.
  raw_017FU3M["raw_{0}".format(count)] = mne.io.read_raw_fieldtrip(
C:\Users\jebe12\AppData\Local\Temp\ipykernel_20540\96999579.py:12: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_R_13. Making it a MISC channel.
  raw_017FU3M["raw_{0}".format(count)] = mne.io.read_raw_fieldtrip(
C:\Users\jebe12\AppData\Local\Temp\ipykernel_20540\96999579.py:12: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_R_02. Making it a MISC channel.
  raw_017FU3M["raw_{0}".format(count)] = mne.io.read_raw_fieldtrip(
C:

In [46]:
raw.info.keys()


dict_keys(['acq_pars', 'acq_stim', 'ctf_head_t', 'description', 'dev_ctf_t', 'dig', 'experimenter', 'utc_offset', 'device_info', 'file_id', 'highpass', 'hpi_subsystem', 'kit_system_id', 'helium_info', 'line_freq', 'lowpass', 'meas_date', 'meas_id', 'proj_id', 'proj_name', 'subject_info', 'xplotter_layout', 'gantry_angle', 'bads', 'chs', 'comps', 'events', 'hpi_meas', 'hpi_results', 'projs', 'proc_history', 'custom_ref_applied', 'sfreq', 'dev_head_t', 'ch_names', 'nchan'])

In [40]:
raw_017FU3M.keys()

dict_keys(['raw_0', 'raw_1', 'raw_2', 'raw_3', 'raw_4', 'raw_5'])

In [50]:
raw_017FU3M["raw_1"].info.keys()

dict_keys(['acq_pars', 'acq_stim', 'ctf_head_t', 'description', 'dev_ctf_t', 'dig', 'experimenter', 'utc_offset', 'device_info', 'file_id', 'highpass', 'hpi_subsystem', 'kit_system_id', 'helium_info', 'line_freq', 'lowpass', 'meas_date', 'meas_id', 'proj_id', 'proj_name', 'subject_info', 'xplotter_layout', 'gantry_angle', 'bads', 'chs', 'comps', 'events', 'hpi_meas', 'hpi_results', 'projs', 'proc_history', 'custom_ref_applied', 'sfreq', 'dev_head_t', 'ch_names', 'nchan'])

### Define variables 

In [19]:



ch_names = raw.ch_names
n_chan = len(ch_names)
n_time_samps = raw.n_times #nsamples
time_secs = raw.times #timepoints set to zero
ch_trials = raw._data
sampling_freq = raw.info['sfreq']
time_duration = (n_time_samps/sampling_freq).astype(float)



print(
      f'The data object has:\n\t{n_time_samps} time samples,'
      f'\n\tand a sample frequency of {sampling_freq} Hz' 
      f'\n\twith a recording duration of {time_duration} seconds.' 
      f'\n\t{n_chan} channels were labeled as \n{ch_names}.'
)

The data object has:
	5312 time samples,
	and a sample frequency of 250.0 Hz
	with a recording duration of 21.248 seconds.
	3 channels were labeled as 
['LFP_Stn_R_1A2A', 'LFP_Stn_R_1B2B', 'LFP_Stn_R_1C2C'].


## 2. Analyze data